In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import cftime
import warnings
xr.set_options(keep_attrs=True)
warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, '../')
import my_utils as myf
import importlib
importlib.reload(myf)

<module 'my_utils' from '/gpfsm/dhome/laroach1/analysis/analysis_modelE-so-sst/published/processing/../my_utils.py'>

In [2]:
c6dir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/historical/'
sspdir = myf.cmip_dir+'ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp245/'

In [3]:
griddir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/piControl/r1i1p1f1/fx/'
garea = xr.open_dataset(griddir+'areacella/gn/v20180824/areacella_fx_GISS-E2-1-G_piControl_r1i1p1f1_gn.nc').areacella
ocnfr = xr.open_dataset(griddir+'sftlf/gn/v20180824/sftlf_fx_GISS-E2-1-G_piControl_r1i1p1f1_gn.nc').sftlf
ocnfr = ocnfr/100.
ocnfr = 1. - ocnfr
garea = garea*ocnfr

In [4]:
def open_cmip(ens):
    
    ds = xr.open_mfdataset(c6dir+ens+'/SImon/siconca/gn/*/siconca_SImon_GISS-E2-1-G_historical_'+ens+'_*.nc')
    
    filestr = 'historical'
    if os.path.isdir(sspdir+ens):
        print('extend with SSP245')
        ds2 = xr.open_mfdataset(sspdir+ens+'/SImon/siconca/gn/*/siconca_SImon_GISS-E2-1-G_ssp245_'+ens+'_*.nc')
        ds = xr.concat([ds,ds2],dim='time').sel(time=slice('1979','2021'))
        filestr = 'hist+ssp245'
    ds['time'] = ds.indexes['time'].to_datetimeindex()
    ds['names'] = ens
    ds = ds.set_coords('names')
    ds = ds.sel(time=slice('1979','2021'))

    return ds, filestr

In [5]:
def get_sia(ds, outfile):
          
    y1 = str(ds.time.values[0])[:4]
    y2 = str(ds.time.values[-1])[:4]
    
    sia_nh = (ds.siconca/100.*garea).where(ds.lat>0).sum(dim=('lat','lon')).to_dataset(name='sia_nh')/1e12
    sia_sh = (ds.siconca/100.*garea).where(ds.lat<0).sum(dim=('lat','lon')).to_dataset(name='sia_sh')/1e12

    ints = ds.siconca.where(ds.siconca>15.)
    ints = ints/ints
    sie_nh = (ints*garea).where(ds.lat>0).sum(dim=('lat','lon')).to_dataset(name='sie_nh')/1e12
    sie_sh = (ints*garea).where(ds.lat<0).sum(dim=('lat','lon')).to_dataset(name='sie_sh')/1e12

    ds = xr.merge([sia_nh, sia_sh, sie_nh, sie_sh])
    
    for var in ds:
        ds[var].attrs['units'] = 'million km^2'

    ds.sia_nh.attrs['long_name'] = 'sea_ice_area_arctic'
    ds.sia_sh.attrs['long_name'] = 'sea_ice_area_antarctic'
    ds.sie_nh.attrs['long_name'] = 'sea_ice_extent_arctic'
    ds.sie_sh.attrs['long_name'] = 'sea_ice_extent_antarctic'
    ds.to_netcdf(myf.processed_dir+'/timeseries/sia/'+outfile+str(y1)+'-'+str(y2)+'.nc')
    
    return ds

In [6]:
runmap = pd.read_csv('../e2.1runmap.csv',sep=',')
myens = runmap['names'].values

In [7]:
for ens in myens:
    ds, filestr = open_cmip(ens)
    ds = get_sia(ds,'siconca_avg_SImon_GISS-E2-1-G_'+filestr+'_'+ens+'_gn_')

extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
